# Tutorial: 阶段 2-3：摘要筛选与全文下载

受众:
- 已完成元数据聚合的用户

前置条件:
- 元数据 CSV 已生成
- 已配置 LLM API Key（可选）

本篇目标:
- 完成摘要筛选并得到过滤后的 CSV
- 批量下载全文文件



## 1. 摘要筛选（LLM）
- 输入：`data_resourse/assets1/elsevier_metadata.csv`
- 输出：`data_resourse/assets1/elsevier_metadata_filtered.csv`

终端执行：
```bash
python -m bensci.metadata_filter_utils
```

> 如果未配置 LLM API Key，会直接复制原 CSV（不筛选）。


In [ ]:
from bensci import metadata_filter_utils as mf

RUN = False  # 改为 True 后执行筛选

if RUN:
    count = mf.filter_metadata()
    print("passed:", count)
else:
    print("Skip running. Output:", mf.TARGET_CSV)



## 2. 全文下载
- 默认根据筛选 CSV 的 DOI 下载
- 默认输出目录：`data_resourse/assets2/`

终端执行：
```bash
python -m bensci.literature_fetcher
```

也可以指定 DOI（多个用逗号分隔）：
```bash
python -m bensci.literature_fetcher --doi "10.xxx/xxxx,10.yyy/yyyy"
```


In [ ]:
from bensci import literature_fetcher as lf
from bensci import config as cfg

RUN = False  # 改为 True 后执行下载

if RUN:
    lf.download_fulltexts()
    print("downloads in:", cfg.ASSETS2_DIR)
else:
    print("Skip running. Output:", cfg.ASSETS2_DIR)



## 3. 查看已下载文件（可选）


In [ ]:
from pathlib import Path
from bensci import config as cfg

path = Path(cfg.ASSETS2_DIR)
if path.exists():
    files = list(path.glob("**/*"))
    print("count:", len(files))
    for item in files[:10]:
        print(item)
else:
    print("not found:", path)


## LLM 参数提示
- 筛选阶段可通过 `METADATA_FILTER_*` 覆盖 Base URL / Chat Path / API Key Env。
- 如果 LLM 返回过慢，可适当提高 `METADATA_FILTER_TIMEOUT` 或降低请求频率。

## 下载顺序说明
- 默认优先顺序由 `LITERATURE_FETCHER_PROVIDER_ORDER` 控制。
- 如果某些 DOI 无法下载，可尝试 `--provider scihub`（需自行评估合规性）。

